# 1번. CNN & RNN

## 1.1 CNN cifar10

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

In [31]:
# seed 고정
def seed_everything(seed):
    #random.seed(seed)
    #os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
seed_everything(123)

In [32]:
# prepare Dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 8
valid_data_size = 10000
trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True,
                                        download=True, transform=transform)
train, validset = torch.utils.data.random_split(trainset, [len(trainset) - valid_data_size, valid_data_size])
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(len(trainloader.dataset),len(validloader.dataset),len(testloader.dataset))

Files already downloaded and verified
Files already downloaded and verified
50000 10000 10000


trainig code

In [33]:
def training_model(model_name,model,train_loader,valid_loader,device,save_name='best'):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                            lr_lambda=lambda epoch: 1.0 if epoch<=5 else 0.94**(epoch-5))
    n_epochs = 100

    train_loss_list , valid_loss_list = [] , []
    best_score = float('inf')

    model.train()
    patience = 0
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data, target in train_loader:
            data , target = data.to(device) , target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        scheduler.step()
        train_loss_list.append(train_loss/len(train_loader.dataset))

        # valid_data
        with torch.no_grad():
            valid_loss = 0.0
            correct_num = 0.0

            for data, target in valid_loader:
                data , target = data.to(device) , target.to(device)
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()*data.size(0)
                predicted_classes = torch.argmax(output, dim=1)
                correct_num += (predicted_classes == target).sum().item()
            valid_loss_list.append(valid_loss/len(valid_loader.dataset))
            correct_num /= len(valid_loader.dataset)
        
            print('Epoch: {} \tTraining Loss: {:.6f} \t valid Loss: {:.6f} \t valid Acc{}'.format(epoch+1, train_loss_list[-1],valid_loss_list[-1],correct_num))
            
        if valid_loss_list[-1] < best_score:
            best_score = valid_loss_list[-1]
            torch.save(model.state_dict(), os.path.join('./model_data/', '{}_{}.pth'.format(model_name,save_name)))
            patience = 0
        else:
            patience += 1
            if patience == 8:
                return train_loss_list , valid_loss_list

def test_model(model_name,model,test_loader,device):
    
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    criterion = nn.CrossEntropyLoss()

    model.eval()

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        
            
    test_loss = test_loss/len(test_loader.dataset)
    print(f'{model_name}=========================================================')
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                str(i), 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    print('==========================================================================')
    print()

AlexNet/VGG/ResNet/DenseNet 불러오기

In [34]:
'''
    [common preprocessing]
        resie 256*256 -> central_crop 224*224  
        mean=[0.485, 0.456, 0.406] std=[0.229, 0.224, 0.225]

    [AlexNet config]
        num_params : 61,100,840

    [VGG19]
        BN은 사용x 후에 Batch Normalization에서 적용
        num_params : 143,667,240

    [ResNet50]
        50 vs 101 vs 152가 성능이 비슷비슷해서 효율이 좋은 ResNet50 사용
        num_params : 25,557,032
    
    [DenseNet161]
        121부터 201까지 있는데 가장 성능이 좋은 161사용 
        특이하게도 layer수가 많아질수록 imageNet 성능이 안좋음
        num_params : 28,681,000
'''

from torchvision.models import alexnet, AlexNet_Weights 
from torchvision.models import vgg19, VGG19_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import densenet161, DenseNet161_Weights

### Model finetuing

기존의 pretrained 모델은 224*224 image입력을 받음

cifar10은 32*32 image임

따라서 max pooling 모두 제거 , adaptivepooling으로 최종 feature size맞추기

ResNet과 DenseNet은 첫번째 pooling만 제거

In [35]:
def load_finetuned_model(model_name,update_conv=False):
    if model_name == 'alex' : 
        model = alexnet(weights=AlexNet_Weights)
        for param in model.features.parameters():
            param.requires_grad = update_conv
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
        return model

    elif model_name == 'vgg' : 
        model = vgg19(weights=VGG19_Weights)
        for param in model.features.parameters():
            param.requires_grad = update_conv
        model.classifier[6] = nn.Linear(4096, 10)
        return model
        

    elif model_name == 'resnet':
        model = resnet50(weights=ResNet50_Weights)
        for param in model.parameters():
            param.requires_grad = update_conv
        model.fc = nn.Linear(model.fc.in_features, 10)
        return model
    
    elif model_name == 'densenet':
        model = densenet161(weights=DenseNet161_Weights)
        for param in model.parameters():
            param.requires_grad = update_conv
        model.classifier = nn.Linear(model.classifier.in_features, 10)
        return model

In [36]:
# train
#model_list = ['alex','vgg','resnet','densenet']
model_list = ['densenet']
acc_dic = dict()
update_conv_token = True
save_name = 'best_true' if update_conv_token else 'best_false'

for model_name in model_list:
    training_dic = {
        'train_loss' : [] , 'valid_loss' : []
    }

    model = load_finetuned_model(model_name,update_conv=update_conv_token)
    #model.load_state_dict(torch.load(os.path.join('./model_data/', '{}_{}.pth'.format(model_name,save_name))))
    model.to(device)
    print(f'{model_name} training start')
    training_dic['train_loss'] , training_dic['valid_loss'] = training_model(model_name,model,trainloader,validloader,device,save_name=save_name)
    print(f'{model_name} training end')
    del model

densenet training start
Epoch: 1 	Training Loss: 1.820886 	 valid Loss: 1.356005 	 valid Acc0.5232
Epoch: 2 	Training Loss: 1.184854 	 valid Loss: 1.021823 	 valid Acc0.6416


KeyboardInterrupt: 

In [ ]:
# test
model_list = ['resnet','densenet']
update_conv_token = True
save_name = 'best_true' if update_conv_token else 'best_false'

for model_name in model_list:
    model = load_finetuned_model(model_name)
    model.load_state_dict(torch.load(os.path.join('./model_data/', '{}_{}.pth'.format(model_name,save_name))))
    model.to(device)
    test_model(model_name,model,testloader,device)

alex=========================================================
Test Loss: 0.586114

Test Accuracy of     0: 82% (820/1000)
Test Accuracy of     1: 80% (809/1000)
Test Accuracy of     2: 73% (733/1000)
Test Accuracy of     3: 68% (683/1000)
Test Accuracy of     4: 63% (639/1000)
Test Accuracy of     5: 83% (834/1000)
Test Accuracy of     6: 87% (871/1000)
Test Accuracy of     7: 72% (729/1000)
Test Accuracy of     8: 88% (888/1000)
Test Accuracy of     9: 93% (931/1000)

Test Accuracy (Overall): 79% (7937/10000)



||test_acc|error_rate|train_loss|valid_loss|epoch|
|------|---|---|---|---|---|
|AlexNet|%|%||||
|AlexNet with conv train|92.37%|7.63%|0.0021|0.0014||
|VGG19|%|%||||
|VGG19 with conv train|87.37%|12.63%|0.0007|0.0005||
|ResNet50|%|%||||
|ResNet50 with conv train|%|%||||
|DenseNet161|%|%||||
|DenseNet161 with conv train|%|%||||